# Hydropower Simulations with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways.
This example demonstrates a few of the options for modeling hydropower generation.

## Dependencies

In [44]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))

"/Users/cbarrows/Documents/repos/Examples"

### Modeling Packages

In [ ]:
using InfrastructureSystems
const IS = InfrastructureSystems
using PowerSystems
const PSY = PowerSystems
using PowerSimulations
const PSI = PowerSimulations
using D3TypeTrees

### Data management packages

In [ ]:
using Dates
using DataFrames

### Optimization packages

In [ ]:
using JuMP
using Cbc # solver
Cbc_optimizer = JuMP.with_optimizer(Cbc.Optimizer, logLevel = 1, ratioGap = 0.5)

### Data
There is a meaningless test dataset assembled in the
[make_hydropower_data.jl](../../script/PowerSimulations_examples/make_hydro_data.jl) script.

In [45]:
include(joinpath(pkgpath,"script/PowerSimulations_examples/make_hydro_data.jl"))

┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = voltage
│   field_value = 1.07
│   valid_range = voltagelimits
│   valid_info.ist_struct = Bus(6, "Bus 6", PowerSystems.PV, -0.24818581963359368, 1.07, (min = 0.94, max = 1.06), 13.8, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("72a7ac7f-6a97-4a9a-a86e-b9108f495129"), nothing))
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/Tjbyn/src/validation.jl:202
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = voltage
│   field_value = 1.062
│   valid_range = voltagelimits
│   valid_info.ist_struct = Bus(7, "Bus 7", PowerSystems.PQ, -0.23335052099164186, 1.062, (min = 0.94, max = 1.06), 13.8, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("3fd4e0bd-e71a-44ee-a1f8-97f06772cf47"), nothing))
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/Tjbyn/src/validation.jl:202
┌ Warnin

## Two PowerSimulations features determine hydropower representation.

### Hydropower `DeviceModel`s

First, the assignment of device formulations to particular device types gives us control
over the representation of devices. This is accomplished by defining `DeviceModel`
instances. For hydro power representations, we have two available generator types in
PowerSystems:

In [42]:
TypeTree(PSY.HydroGen)

HydroGen
├──HydroFix
└──HydroDispatch

And in PowerSimulations, we have several available formulations that can be applied to
the hydropower generation devices:

In [ ]:
TypeTree(PSI.AbstractHydroFormulation)

Let's see what some of the different combinations create. First, let's apply the
`HydroDispatchRunOfRiver` formulation to the `HydroDispatch` generators, and the
`HydroFixed` formulation to `HydroFix` generators.

In [ ]:
devices = Dict{Symbol,DeviceModel}(:Hyd1 => DeviceModel(HydroDispatch, HydroDispatchRunOfRiver),
    :Hyd2 => DeviceModel(HydroFix, HydroFixed),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 2)

Now we can see the resulting JuMP model:

In [ ]:
op_problem.psi_container.JuMPmodel

Next, let's apply the `HydroDispatchReservoirFlow` formulation to the `HydroDispatch` generators, and the
`HydroDispatchRunOfRiver` formulation to `HydroFix` generators.

In [ ]:
devices = Dict{Symbol,DeviceModel}(:Hyd1 => DeviceModel(HydroDispatch, HydroDispatchReservoirFlow),
    :Hyd2 => DeviceModel(HydroFix, HydroDispatchRunOfRiver),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 2)

And, the resulting JuMP model:

In [ ]:
op_problem.psi_container.JuMPmodel

Next, let's apply the `HydroDispatchReservoirStorage` formulation to the `HydroDispatch` generators, and the
`HydroDispatchRunOfRiver` formulation to `HydroFix` generators.

In [ ]:
devices = Dict{Symbol,DeviceModel}(:Hyd1 => DeviceModel(HydroDispatch, HydroDispatchReservoirStorage),
    :Hyd2 => DeviceModel(HydroFix, HydroDispatchRunOfRiver),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 2)

-

In [ ]:
op_problem.psi_container.JuMPmodel

Finally, let's see the `HydroCommitmentReservoirFlow` formulation applied to the `HydroDispatch` generators, and the
`HydroDispatchRunOfRiver` formulation to `HydroFix` generators.

In [ ]:
devices = Dict{Symbol,DeviceModel}(:Hyd1 => DeviceModel(HydroDispatch, HydroCommitmentReservoirStorage),
    :Hyd2 => DeviceModel(HydroFix, HydroDispatchRunOfRiver),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 2)

-

In [ ]:
op_problem.psi_container.JuMPmodel

### Multi-Stage `SimulationSequence`

UC model template

In [ ]:
devices = Dict(:Generators => DeviceModel(ThermalStandard, ThermalBasicUnitCommitment),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroDispatch => DeviceModel(HydroDispatch, HydroDispatchRunOfRiver),
)
template_uc = OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

ED model template

In [ ]:
devices = Dict(:Generators => DeviceModel(ThermalStandard, ThermalDispatchNoMin),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :ILoads => DeviceModel(InterruptibleLoad, DispatchablePowerLoad),
    :HydroDispatch => DeviceModel(HydroDispatch, HydroDispatchReservoirFlow),
)
template_ed = OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

Simulaiton setup

In [ ]:
stages_definition = Dict("UC" => Stage(GenericOpProblem, template_uc, c_sys5_hy, Cbc_optimizer),
    "ED" => Stage(GenericOpProblem, template_ed, c_sys5_hy_ed, Cbc_optimizer),
)

sequence = SimulationSequence(order = Dict(1 => "UC", 2 => "ED"),
    intra_stage_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24)),
    horizons = Dict("UC" => 24, "ED" => 12),
    intervals = Dict("UC" => Hour(24), "ED" => Hour(1)),
    feed_forward = Dict(("ED", :devices, :Generators) => SemiContinuousFF(binary_from_stage = Symbol(PSI.ON),
            affected_variables = [Symbol(PSI.REAL_POWER)],
        ),
        ("ED", :devices, :HydroDispatch) => IntegralLimitFF(variable_from_stage = Symbol(PSI.REAL_POWER),
            affected_variables = [Symbol(PSI.REAL_POWER)],
        ),
    ),
    cache = Dict("ED" => [TimeStatusChange(Symbol(PSI.ON))]),

    ini_cond_chronology = Dict("UC" => Consecutive(), "ED" => Consecutive()),

)
sim = Simulation(name = "hydro",
    steps = 2,
    step_resolution = Hour(24),
    stages = stages_definition,
    stages_sequence = sequence,
    simulation_folder = file_path,
    verbose = true,
)

build!(sim)


sim_results = execute!(sim; verbose = true)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*